# Class: Single-Index Model

In [2]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import copy           #initializing without "copy" (call by reference) let to wrong regression results (neg R2 etc). hence, call by value vs reference matters (unmutable vs mutable)
from scipy.stats import spearmanr #for p-value of correlation matrix of residuals 

class SingleIndexModel():
    
    def __init__(self, r, f1):  
        #input: r: pandas data frame
        #input: f1: pandas data frame
        self.r = copy.copy(r)                 #alternatively: pd.DataFrame(r).copy()
        self.f1 = copy.copy(f1)               #alternatively: pd.DataFrame(f1).copy() 
        self.eps = copy.copy(r)               #alternatively: pd.DataFrame(r).copy()
        self.Rho = pd.DataFrame               #correlation matrix of residuals
        
        self.alpha = np.zeros((2,r.shape[1])) #point estimate // t-vlue
        self.beta = np.zeros((2,r.shape[1]))  #point estimate // t-vlue
        
        self.adjR2       = np.zeros((1,r.shape[1]))
        
        
    def fit_linReg(self):
        X = self.f1
        X =sm.add_constant(X)
        
        for i in range(0,self.r.shape[1]): 
            y = self.r.iloc[:,i]
            
            linReg = sm.GLS(y,X)
            output = linReg.fit()
                      
            self.eps.iloc[:,i] = y-output.fittedvalues
            
            self.Rho = self.eps.corr(method="spearman") #correlation among firm-specific shocks
            self.Rho_sc, self.Rho_pvl_scp = spearmanr(self.eps)
            
            self.alpha[0,i] = output.params[0]
            self.alpha[1,i] = output.tvalues[0]
            
            self.beta[0,i]  = output.params[1]
            self.beta[1,i]  = output.tvalues[1]
            
            self.adjR2[0,i] = output.rsquared_adj 
            
            
             